We solve the wave equation (in Maxwell form) for speed $c=2$, with homogeneous Dirichlet conditions on the first variable. 

In [ ]:
include("../FNC.jl")
m = 200
x,Dx = FNC.diffmat2(m,[-1,1]);

The boundary values of $u$ are given to be zero, so they are not unknowns in the ODEs we solve. Instead they are added or removed as necessary. 

In [ ]:
chop(u) = u[2:m];
extend(v) = [0;v;0];

The following function computes the time derivative of the system at interior points.

In [ ]:
dwdt = function (w,c,t)
    u = extend(w[1:m-1])
    z = w[m:2*m]
    dudt = Dx*z
    dzdt = c^2*(Dx*u)
    return [ chop(dudt); dzdt ]
    end;

Our initial condition is a single hump for $u$.

In [ ]:
u_init = @.exp(-100*(x)^2)
z_init = -u_init
w_init = [ chop(u_init); z_init ];  

Because the wave equation is hyperbolic, we can use a nonstiff explicit solver.

In [ ]:
using DifferentialEquations
IVP = ODEProblem(dwdt,w_init,(0.,2.),2)
sol = solve(IVP,RK4());

We extract the original $u$ and $z$ variables from the results,  adding in the zeros at the boundaries for $u$.

In [ ]:
n = length(sol.t)-1
U = [ zeros(1,n+1); sol[1:m-1,:]; zeros(1,n+1) ];
Z = sol[m:2*m,:];

We plot the results for the original $u$ variable. 

In [ ]:
using Plots
@gif for (i,t) = enumerate(sol.t) 
    plot(x,U[:,i],layout=(1,2),subplot=1,
        xaxis=("x"),yaxis=([-1,1],"u(x,t)"),    
        title="Wave equation, t=$(round(t,digits=3))",leg=:none )
    plot!(x,sol.t,U',subplot=2,xlabel="x",ylabel="t",title="Space-time view",leg=:none)
end

The original hump breaks into two pieces of different amplitudes. Each travels with speed $c=2$, and they pass through one another without interference. When a hump encounters a boundary, it is perfectly reflected, but with inverted shape. At time $t=2$ the exact solution looks just like the initial condition.